## product.csv 생성

In [5]:
import pandas as pd
from datetime import datetime
import uuid

seed = pd.read_csv("./dataset/seed_with_category.csv")

now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

products = []

for _, row in seed.iterrows():
    products.append({
        "product_code": str(uuid.uuid4())[:12],  # 짧은 UUID
        "name": row["name"],
        "description": row["description"],
        "price": row["base_price"],
        "status": "ACTIVE",
        "category_id": int(row["category_id"]),
        "thumbnail_image_url": None,
        "brand": None,
        "created_at": now,
        "updated_at": now
    })

products_df = pd.DataFrame(products)
products_df.to_csv("./dataset/catalog_products.csv", index=False)

## product_skus.csv 생성

In [6]:
sku_df = pd.read_csv("./dataset/sku_with_stock.csv")
sku_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 483057 entries, 0 to 483056
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   sku_id               483057 non-null  object
 1   external_product_id  483057 non-null  int64 
 2   price                483057 non-null  int64 
 3   options              483057 non-null  object
 4   stock                483057 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 18.4+ MB


In [7]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103506 entries, 0 to 103505
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   product_code         103506 non-null  object
 1   name                 103506 non-null  object
 2   description          103506 non-null  object
 3   price                103506 non-null  int64 
 4   status               103506 non-null  object
 5   category_id          103506 non-null  int64 
 6   thumbnail_image_url  0 non-null       object
 7   brand                0 non-null       object
 8   created_at           103506 non-null  object
 9   updated_at           103506 non-null  object
dtypes: int64(2), object(8)
memory usage: 7.9+ MB


In [8]:
# products_df에 external_product_id 붙이기
products_df["external_product_id"] = seed["external_product_id"]

# 매핑 확인
products_df.head()

,product_code,name,description,price,status,category_id,thumbnail_image_url,brand,created_at,updated_at,external_product_id
0,b98f8b6d-f01,Tailgating Pros American Flag Whitewashed Rust...,Tailgating Pros Cornhole Set w/ Bean Bags Incl...,56890,ACTIVE,58,None,None,2025-12-12 01:57:00,2025-12-12 01:57:00,2130707
1,3b15038a-0b1,Venus Aqua Aquarium Air Pump Fishing Supplier ...,<b><p>Venus Aqua Aquarium Air Pump Fishing Sup...,81743,ACTIVE,1,None,None,2025-12-12 01:57:00,2025-12-12 01:57:00,2746668
2,0b21f608-a69,Stylish Tempered Glass backsplash – Glass Kitc...,<p>This 4 mm thick toughened glass BACKSPLASH ...,139027,ACTIVE,107,None,None,2025-12-12 01:57:00,2025-12-12 01:57:00,1611407
3,4baa8989-10b,TODO DEALS 9H Unbreakable Car Music System Scr...,ULTIMATE PROTECTION AND USABILITY: Our Premium...,48874,ACTIVE,17,None,None,2025-12-12 01:57:00,2025-12-12 01:57:00,2785306
4,e07db0f5-ae8,Beck Arnley 041-0857 Oil Filter,Steering Wheel Air Bag,116722,ACTIVE,11,None,None,2025-12-12 01:57:00,2025-12-12 01:57:00,1032025


In [9]:
merged = sku_df.merge(
    products_df[["external_product_id", "product_code"]],
    on="external_product_id",
    how="left"
)

merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 483057 entries, 0 to 483056
Data columns (total 6 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   sku_id               483057 non-null  object
 1   external_product_id  483057 non-null  int64 
 2   price                483057 non-null  int64 
 3   options              483057 non-null  object
 4   stock                483057 non-null  int64 
 5   product_code         483057 non-null  object
dtypes: int64(3), object(3)
memory usage: 22.1+ MB


In [10]:
product_skus = merged[[
    "sku_id",
    "product_code",
    "price",
    "options"
]].copy()

product_skus["created_at"] = now

product_skus.to_csv("./dataset/catalog_product_skus.csv", index=False)
print("product_skus rows:", len(product_skus))

product_skus rows: 483057


## option_groups & options 생성

In [11]:
{"Color": "Black", "Size": "M"}

{'Color': 'Black', 'Size': 'M'}

In [12]:
import json

option_groups = []
options = []

group_id = 1
option_id = 1

# SKU를 product_code 기준으로 그룹화
for product_code, rows in merged.groupby("product_code"):
    # 모든 옵션 dict 수집
    option_dicts = []
    for o in rows["options"]:
        try:
            opt = json.loads(o.replace("'", "\"")) if isinstance(o, str) else {}
        except:
            opt = {}
        option_dicts.append(opt)

    # 모든 옵션 key 수집
    keys = set()
    for od in option_dicts:
        keys |= set(od.keys())

    # 옵션 그룹 생성
    for ordering, key in enumerate(sorted(keys)):
        option_groups.append({
            "id": group_id,
            "product_code": product_code,
            "name": key,
            "ordering": ordering,
            "created_at": now,
            "updated_at": now
        })

        # 옵션 값들 dedupe
        values = sorted(set(od.get(key) for od in option_dicts if key in od))

        for order2, value in enumerate(values):
            options.append({
                "id": option_id,
                "option_group_id": group_id,
                "name": value,
                "additional_price": 0,
                "ordering": order2,
                "created_at": now,
                "updated_at": now
            })
            option_id += 1

        group_id += 1

option_groups_df = pd.DataFrame(option_groups)
options_df = pd.DataFrame(options)

option_groups_df.to_csv("./dataset/catalog_option_groups.csv", index=False)
options_df.to_csv("./dataset/catalog_options.csv", index=False)

print("option_groups:", len(option_groups_df))
print("options:", len(options_df))

option_groups: 84366
options: 267290


## inventory DB csv 생성

In [13]:
inventory_rows = []

for _, row in sku_df.iterrows():
    inventory_rows.append({
        "sku_id": row["sku_id"],
        "total_stock": int(row["stock"]),
        "reserved_stock": 0,
        "created_at": now,
        "updated_at": now
    })

inventory_df = pd.DataFrame(inventory_rows)
inventory_df.to_csv("./dataset/inventory.csv", index=False)

print("inventory rows:", len(inventory_df))

inventory rows: 483057
